In [1]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import signal
import cv2
from PIL import Image
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision.transforms as transforms

In [2]:
frames = []

def extract_frames(video_path, interval_seconds):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file:", video_path)
        return []

    fps = cap.get(cv2.CAP_PROP_FPS)
    interval_frames = int(fps * interval_seconds)
    frame_count = 0
    extracted_frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % interval_frames == 0:
            extracted_frames.append(frame)

    cap.release()
    return extracted_frames

for i in range(1, 9):  # Corrected the range to iterate from 1 to 9
    video_path = '/content/drive/MyDrive/Internshala/Videos/'+str(i)+'.mp4'
    interval_seconds = 10

    extracted_frames = extract_frames(video_path, interval_seconds)
    frames.extend(extracted_frames)

frames_array = np.array(frames)

In [3]:
images_edge = []  # Creating an empty list to store edge images
for i in range(len(frames)):
  gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
  blurred = cv2.GaussianBlur(gray, (5,5), 0)
  edges = cv2.Canny(blurred, 30, 80)
  images_edge.append(edges)  # Appending each edge image to the list

images_edge = np.array(images_edge)
len(images_edge)

387

In [ ]:
# Read the CSV file where data is stored for training
df = pd.read_csv('/content/drive/MyDrive/internship.csv')

# Split the data into training, cross-validation, and test sets
images_train = images_edge[0:235]  # Assuming the first column contains image paths
labels_train = df.iloc[1:236, 1:].values  # Convert DataFrame to numpy array
images_cv = images_edge[235:315]  # Assuming the first column contains image paths
labels_cv = df.iloc[236:316, 1:].values  # Convert DataFrame to numpy array
images_test = images_edge[315:]  # Assuming the first column contains image paths
labels_test = df.iloc[316:388, 1:].values  # Convert DataFrame to numpy array

# Convert your data to PyTorch tensors
images_tensor_train = torch.tensor(images_train)  # Convert image data to PyTorch tensor
labels_tensor_train = torch.tensor(labels_train)  # Convert additional features data to PyTorch tensor
images_tensor_cv = torch.tensor(images_cv)  # Convert image data to PyTorch tensor
labels_tensor_cv = torch.tensor(labels_cv)  # Convert additional features data to PyTorch tensor
images_tensor_test = torch.tensor(images_test)  # Convert image data to PyTorch tensor
labels_tensor_test = torch.tensor(labels_test)  # Convert additional features data to PyTorch tensor

print(len(images_train), len(images_cv), len(images_test))
print(len(labels_train), len(labels_cv), len(labels_test))
# Create a combined dataset if needed
# For example, if you want to combine image data and additional features data:
combined_dataset_train = torch.utils.data.TensorDataset(images_tensor_train, labels_tensor_train)
combined_dataset_cv = torch.utils.data.TensorDataset(images_tensor_cv, labels_tensor_cv)
combined_dataset_test = torch.utils.data.TensorDataset(images_tensor_test, labels_tensor_test)

In [ ]:
# Define the custom model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # Updated in_channels to 1 for grayscale images
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.output_branch1 = nn.Sequential(
            nn.Linear(256 * 11 * 20, 512),  # Adjusted to match the size of the flattened feature map
            nn.ReLU(),
            nn.Linear(512, 3)
        )

        self.output_branch2 = nn.Sequential(
            nn.Linear(256 * 11 * 20, 512),  # Adjusted to match the size of the flattened feature map
            nn.ReLU(),
            nn.Linear(512, 3)
        )

        self.output_branch3 = nn.Sequential(
            nn.Linear(256 * 11 * 20, 512),  # Adjusted to match the size of the flattened feature map
            nn.ReLU(),
            nn.Linear(512, 3)
        )

        self.output_branch4 = nn.Sequential(
            nn.Linear(256 * 11 * 20, 512),  # Adjusted to match the size of the flattened feature map
            nn.ReLU(),
            nn.Linear(512, 3)
        )

    def forward(self, x):
        x = self.features(x)
        print("Size of feature map before flattening:", x.size())  # Add this line for debugging
        x = torch.flatten(x, 1)
        print("Size of flattened feature map:", x.size())  # Add this line for debugging
        out1 = self.output_branch1(x)
        out2 = self.output_branch2(x)
        out3 = self.output_branch3(x)
        out4 = self.output_branch4(x)
        return out1, out2, out3, out4


# Assuming you have input image data (images_train) and corresponding labels
# Convert your data to PyTorch tensors
images_tensor = torch.tensor(images_train).float().unsqueeze(1)  # Convert grayscale images to PyTorch tensor and make it float
labels_tensor = torch.tensor(labels_train).long()  # Convert labels to PyTorch tensor and ensure it's a long tensor

# Create a combined dataset
combined_dataset = TensorDataset(images_tensor, labels_tensor)

# Create a data loader
train_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)

# Instantiate the model
model = CustomModel()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs1, outputs2, outputs3, outputs4 = model(inputs)
        loss1 = criterion(outputs1, labels[:, 0])  # Adjusted to select the appropriate target for each output branch
        loss2 = criterion(outputs2, labels[:, 1])  # Adjusted to select the appropriate target for each output branch
        loss3 = criterion(outputs3, labels[:, 2])  # Adjusted to select the appropriate target for each output branch
        loss4 = criterion(outputs4, labels[:, 3])  # Adjusted to select the appropriate target for each output branch
        total_loss = loss1 + loss2 + loss3 + loss4

        # Backward pass and optimize
        total_loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += total_loss.item()
        if i % 10 == 9:  # Print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

# Save the model's state dictionary
# torch.save(model.state_dict(), 'custom_model_weights.pth')


# Define the file path where you want to save the model weights
file_path = '/content/drive/MyDrive/Colab Notebooks/contweights.pth'

# Save the model's state dictionary at the specified file path
torch.save(model.state_dict(), file_path)

In [ ]:
# Define the custom model class
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.output_branch1 = nn.Sequential(
            nn.Linear(256 * 11 * 20, 512),
            nn.ReLU(),
            nn.Linear(512, 3)
        )

        self.output_branch2 = nn.Sequential(
            nn.Linear(256 * 11 * 20, 512),
            nn.ReLU(),
            nn.Linear(512, 3)
        )

        self.output_branch3 = nn.Sequential(
            nn.Linear(256 * 11 * 20, 512),
            nn.ReLU(),
            nn.Linear(512, 3)
        )

        self.output_branch4 = nn.Sequential(
            nn.Linear(256 * 11 * 20, 512),
            nn.ReLU(),
            nn.Linear(512, 3)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        out1 = self.output_branch1(x)
        out2 = self.output_branch2(x)
        out3 = self.output_branch3(x)
        out4 = self.output_branch4(x)
        return out1, out2, out3, out4

# Assuming you have input images (images_test)
images_tensor = torch.tensor(images_test).float().unsqueeze(1)  # Convert grayscale images to PyTorch tensor and make it float

# Define the dataset and data loader for prediction
val_dataset = TensorDataset(images_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load the trained model
model = CustomModel()
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/contweights.pth'))
model.eval()  # Set the model to evaluation mode

# Make predictions on the test dataset
predictions = []
with torch.no_grad():
    for batch_data in val_loader:
        # Get inputs from the current batch
        inputs = batch_data[0]

        # Forward pass
        outputs1, outputs2, outputs3, outputs4 = model(inputs)
        preds1 = torch.argmax(outputs1, dim=1)
        preds2 = torch.argmax(outputs2, dim=1)
        preds3 = torch.argmax(outputs3, dim=1)
        preds4 = torch.argmax(outputs4, dim=1)
        batch_preds = torch.stack((preds1, preds2, preds3, preds4), dim=1)
        predictions.append(batch_preds)

# Convert predictions to a numpy array
predictions = torch.cat(predictions).numpy()

# Display predictions
print(predictions)